In [6]:
import pandas as pd
import sqlite3

In [2]:
main = pd.read_csv("main.csv")

In [11]:
net_property = pd.read_csv("net_property.csv")

In [4]:
type_property = pd.read_csv("type_property.csv")

In [8]:
# Устанавливаем подключение к базе данных SQLite
conn = sqlite3.connect('indicators_db.sqlite3')

# Выполняем SQL-запрос и считываем результат в датафрейм
query = "SELECT * FROM org_indicators_2019"
df_2019 = pd.read_sql_query(query, conn)
query = "SELECT * FROM org_indicators_2020_2022"
df_2020 = pd.read_sql_query(query, conn)

def collapse_columns(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.to_series().apply(lambda x: "_".join([str(i) for i in x]))
    return df

# Закрываем соединение с базой данных
conn.close()

df_2019 = df_2019.pivot(index="ID", values="VALUE", columns=["YEAR", "INDICATOR_NAME"])
df_2019 = df_2019.reset_index()
df_2019 = collapse_columns(df_2019)
df_2019 = df_2019.rename(columns={"ID_": "ID"}) 
df_2020 = df_2020.pivot(index="ID", values="VALUE", columns=["YEAR", "INDICATOR_NAME"])
df_2020 = df_2020.reset_index()
df_2020 = collapse_columns(df_2020)
df_2020 = df_2020.rename(columns={"ID_": "ID"}) 


In [9]:
df_2021_money = df_2020[["ID", "2021_Выручка"]]

In [16]:
df_5 = pd.merge(main, net_property, how="outer")
df_5 = pd.merge(df_5, type_property, how="outer")
df_5 = pd.merge(df_5, df_2021_money, how="outer")

In [17]:
df_5 = df_5.drop_duplicates()

In [18]:
df_5_net = df_5[(df_5['TypeObject']=='кафе') & (df_5['is_actual']==True) & (df_5['IsNetObject']=='да')]

In [19]:
df_5_notnet = df_5[(df_5['TypeObject']=='кафе') & (df_5['is_actual']==True) & (df_5['IsNetObject']=='нет')]

In [58]:
df_5_net["2021_Выручка"].sum() - df_5_notnet["2021_Выручка"].sum()

-9165261214.0

In [100]:

df_2020_check = df_2020[["ID", "2021_Проверок", "2022_Проверок"]]

In [101]:
df_6 = pd.merge(main[["ID", "Name", "Address"]], df_2020_check, how="left")

In [102]:
columns_to_check = ["2021_Проверок", "2022_Проверок"]

In [104]:
df_6 = df_6[((df_6["2021_Проверок"].isna()) & (df_6["2022_Проверок"].isna())) |
        ((df_6["2021_Проверок"].isna()) & (df_6["2022_Проверок"] == 0)) |
    ((df_6["2021_Проверок"] == 0) & (df_6["2022_Проверок"].isna()) |
 ((df_6["2021_Проверок"] == 0) & (df_6["2022_Проверок"] == 0)))]


In [93]:
df_6 = df_6.sort_values(["Address", "ID"], ascending=[True, True])

In [94]:
result_df = df_6.drop(columns=["2021_Проверок", "2022_Проверок"])

In [95]:
result_df.columns = ['Идентификатор', 'Наименование', 'Адрес']
result_df = result_df.drop_duplicates()

In [90]:
result_df.to_csv("df_6.csv", index=False)